# Monte Carlo Default Simulation

We aim to estimate the expected loss up to time T:

$$E[L_T] = E\left[\sum_{i=1}^{N} \ell_i \cdot 1\{T_i \leq T\}\right]$$

Where:

- $\ell_i$ represents the loss amount from default $i$
- $T_i$ represents the default time for firm $i$
- $1\{T_i \leq T\}$ is the indicator function for defaults occurring before time $T$

## Model Components

### Firm Intensity Process
Each firm's default intensity is modeled as:

$$\lambda_t^i = X_i + \sum_{j=1}^{J} w_{ij} Y_t^j$$

Where:
- $X_i$ is the idiosyncratic component for firm $i$
- $Y_t^j$ is the sectoral intensity component
- $w_{ij}$ are loading factors representing sensitivity to sectoral factors

### Sectoral Intensity Dynamics with Contagion
Each sector follows a process with mean reversion, volatility, and contagion effects:

$$dY_t^j = \kappa_j(\theta_j - Y_t^j)dt + \sqrt{Y_t^j}\sum_{k=1}^{J}\sigma_{jk}dW_t^k + \sum_{k=1}^{J}\delta_{jk}dL_t^k$$

Where:
- $\kappa_j$ is the mean reversion speed
- $\theta_j$ is the long-term mean level
- $\sigma_{jk}$ are volatility parameters
- $W_t^k$ are standard Brownian motions
- $\delta_{jk}$ are contagion coefficients
- $L_t^k$ are loss processes for each sector


## In this notebook we attempt to use the Jump Approximation Methods as discussed in *Reducing Bias in Event Time Simulations via Measure Changes (Giesecke, Shkolnik (2018))*

In [1]:
import numpy as np
import matplotlib.pyplot as plt

RNG = np.random.default_rng(42)

In [2]:
def draw_params(N=100, J=5):

    X = RNG.uniform(0.01, 0.05, size=N)                      # idio intensities

    w0 = RNG.uniform(0,1,(N,J))
    W = w0 / w0.sum(axis=1, keepdims=True)                   # exposures

    ell = np.ones(N)                                         # LGD
    
    kappa = RNG.uniform(0.5,1.5,size=J)
    theta = RNG.uniform(0.05,0.25,size=J)
    
    sigma = np.zeros((J,J))
    for j in range(J):
        s = RNG.uniform(0.1,0.2)
        sigma[j,j] = min(s, np.sqrt(2*kappa[j]*theta[j])) # enforce feller condn 
    
    sigma[sigma==0] = 0.05                                   # off-diag
    
    delta = RNG.uniform(0,0.1,(J,J))
    
    # Brownian corr matrix
    rho = 0.3
    Corr = np.eye(J)*1 + (1-np.eye(J))*rho
    
    return dict(N=N, J=J, X=X, W=W, ell=ell,
                kappa=kappa, theta=theta, sigma=sigma, delta=delta,
                Corr=Corr)


In [3]:
params = draw_params()
print(params)

{'N': 100, 'J': 5, 'X': array([0.04095824, 0.02755514, 0.04434392, 0.03789472, 0.01376709,
       0.04902489, 0.04044559, 0.04144257, 0.01512455, 0.02801544,
       0.02483192, 0.0470706 , 0.0357546 , 0.04291046, 0.02773657,
       0.01908955, 0.03218339, 0.01255269, 0.04310525, 0.03526658,
       0.04032351, 0.02418104, 0.04882792, 0.04572484, 0.04113534,
       0.01778555, 0.02866884, 0.01175215, 0.01617158, 0.03732196,
       0.03979049, 0.04870039, 0.02303301, 0.02481839, 0.02878223,
       0.01757885, 0.01519686, 0.0290282 , 0.01907637, 0.03679256,
       0.02748608, 0.04330713, 0.0380106 , 0.02249467, 0.04329039,
       0.04219057, 0.02549914, 0.02153312, 0.03729982, 0.0155901 ,
       0.01799633, 0.01029449, 0.04147698, 0.03659403, 0.03820662,
       0.04122916, 0.02835663, 0.03274965, 0.01559188, 0.0145812 ,
       0.03673612, 0.02884385, 0.03260944, 0.04059995, 0.03538873,
       0.03214318, 0.03236829, 0.022158  , 0.01123271, 0.0274687 ,
       0.01858339, 0.02634115, 0.04413

In [24]:
import numpy as np

def simulate_plain_jam(params, T=1.0, dt_max=0.0001, seed=None):
    """
    Plain JAM simulation for portfolio credit loss with CIR contagion.
    Returns (total_loss, likelihood_weight).
    """
    if seed is not None:
        np.random.seed(seed)

    N, J = params['N'], params['J']
    X = params['X']
    w = params['W']
    losses = params['ell']
    kappa = params['kappa']
    theta = params['theta']
    sigma = params['sigma']
    delta = params['delta']

    L_chol = np.linalg.cholesky(params['Corr'])
    
    # Initial sector intensities
    Y = theta.copy()

    # keep track of alive firms
    alive = np.arange(N)

    t = 0.0
    L_weight = 1.0
    total_loss = 0.0

    while t < T and alive.size > 0:
        # compute firmwise current intensities
        lambdas = X[alive] + (w[alive] @ Y)
        Lambda_q = lambdas.sum()
        if Lambda_q <= 0:
            break

        Delta = np.random.exponential(1.0 / Lambda_q)
        t_new = t + Delta
        if t_new > T:
            break

        # simulate sector intnesities sdes
        m = max(1, int(np.ceil(Delta / dt_max)))
        dt = Delta / m
        integral_p = 0.0
        t_sub = t
        for _ in range(m):
            # basic euler‐maruyama update
            z = np.random.randn(J)
            dW = L_chol @ (np.sqrt(dt) * z)
            for j in range(J):
                diffusion_term = np.sum(sigma[j, :] * np.sqrt(np.maximum(Y, 0.0)) * dW)
                dY = kappa[j] * (theta[j] - Y[j]) * dt + diffusion_term
                Y[j] = max(Y[j] + dY, 0.0)

            lambdas_sub = X[alive] + (w[alive] @ Y)
            integral_p += lambdas_sub.sum() * dt
            t_sub += dt

        # compute instentiies at t_new
        lambdas = X[alive] + (w[alive] @ Y)

        # choose defaulting firm
        probs = lambdas / Lambda_q
        probs = probs / probs.sum()  # normalise for np.random.choice to work stably
        i_rel = np.random.choice(len(alive), p=probs)
        firm = alive[i_rel]
        lambda_i = lambdas[i_rel]

        # compute the likelihood weight component
        area_Q = Lambda_q * Delta
        G = np.exp(area_Q - integral_p)
        L_weight *= G

        # now we apply the jumps
        # assume each firm has a primary sector k, takie the sector with highest loading
        k = np.argmax(w[firm])
        Y += delta[:, k]

        # record loss and remove firm
        total_loss += losses[firm]
        alive = np.delete(alive, i_rel)

        t = t_new

    return total_loss, L_weight


In [38]:
import numpy as np
from numba import jit, prange
import time

@jit(nopython=True, cache=False)
def evolve_cir(Y, dt, kappa, theta, sigma, L_chol, z):
    dW = L_chol @ z
    J = len(Y)
    new_Y = np.zeros(J)
    for j in range(J):
        drift = kappa[j] * (theta[j] - Y[j]) * dt
        diffusion = np.sum(sigma[j, :] * np.sqrt(np.maximum(Y, 0.0)) * dW)
        new_Y[j] = max(Y[j] + drift + diffusion, 0.0)
    return new_Y

@jit(nopython=True, cache=False)
def simulate_path(X, w, losses, kappa, theta, sigma, delta, L_chol, 
                          Y_init, T, dt_max, rng_state):

    np.random.seed(rng_state)
    N, J = w.shape
    
    Y = Y_init.copy()
    alive_mask = np.ones(N, dtype=np.bool_)
    alive_count = N
    t = 0.0
    L_weight = 1.0
    total_loss = 0.0
    
    # precompute firm-sector mapping
    firm_sectors = np.argmax(w, axis=1)
    
    while t < T and alive_count > 0:
        alive_indices = np.where(alive_mask)[0]

        lambdas = X[alive_indices] + np.sum(w[alive_indices] * Y, axis=1)
        Lambda_q = np.sum(lambdas)
        
        if Lambda_q <= 1e-12:
            break
            
        # sample jump time
        Delta = np.random.exponential(1.0 / Lambda_q)
        t_new = t + Delta
        
        if t_new > T:
            break


        m = max(1, int(np.ceil(Delta / dt_max)))
        dt = Delta / m
        integral_p = 0.0
        
        for _ in range(m):
            z = np.random.randn(J) * np.sqrt(dt)
            Y = evolve_cir(Y, dt, kappa, theta, sigma, L_chol, z)
            
            current_lambdas = X[alive_indices] + np.sum(w[alive_indices] * Y, axis=1)
            integral_p += np.sum(current_lambdas) * dt
        
        lambdas = X[alive_indices] + np.sum(w[alive_indices] * Y, axis=1)
        
        probs = lambdas / Lambda_q
        probs = probs / np.sum(probs) # normalize for stable samplng
        # sample which firm defaulted
        cumsum_probs = np.cumsum(probs)
        u = np.random.random()
        i_rel = np.searchsorted(cumsum_probs, u)
        i_rel = min(i_rel, len(alive_indices) - 1)
        
        firm = alive_indices[i_rel]
        lambda_i = lambdas[i_rel]
        
        # compute likelihood weight
        area_Q = Lambda_q * Delta
        G = np.exp(area_Q - integral_p)
        L_weight *= G
        
        # now we apply the contagion jumps
        k = firm_sectors[firm]
        Y += delta[:, k]
        Y = np.maximum(Y, 0.0)
        
        total_loss += losses[firm]
        alive_mask[firm] = False
        alive_count -= 1
        
        t = t_new
    
    return total_loss, L_weight

@jit(nopython=True, parallel=True, cache=True)
def monte_carlo(X, w, losses, kappa, theta, sigma, delta, L_chol, 
                        Y_init, T, dt_max, M, base_seed):

    results = np.zeros((M, 2))  # [total_loss, L_weight]
    
    for i in prange(M):
        seed = base_seed + i
        total_loss, L_weight = simulate_path(
            X, w, losses, kappa, theta, sigma, delta, L_chol, Y_init, T, dt_max, seed)
        results[i, 0] = total_loss
        results[i, 1] = L_weight
    
    return results

class OptimizedJAMSimulator:
    
    def __init__(self, params, T=1.0, dt_max=0.0001):
        self.params = params
        self.N = params['N']
        self.J = params['J']
        self.T = T
        self.dt_max = dt_max
        
        self._setup_parameters()
        
    def _setup_parameters(self):
        self.X = self.params['X']
        self.w = self.params['W']
        self.losses = self.params['ell']
        self.kappa = self.params['kappa']
        self.theta = self.params['theta']
        self.sigma = self.params['sigma']
        self.delta = self.params['delta']
        
        # Cholesky decomposition for correlated Brownian motion
        self.L_chol = np.linalg.cholesky(self.params['Corr'])
        
        # initial sector intensities - start at theta ie long term mean, for stability
        self.Y_init = self.theta.copy()
    
    def run_simulation(self, M=10000, use_parallel=True):
        start_time = time.time()
        
        if use_parallel:
            results = monte_carlo(
                self.X, self.w, self.losses, self.kappa, self.theta, 
                self.sigma, self.delta, self.L_chol, self.Y_init,
                self.T, self.dt_max, M, 0
            )
        else:
            results = np.zeros((M, 2))
            for i in range(M):
                total_loss, L_weight = simulate_path(
                    self.X, self.w, self.losses, self.kappa, self.theta,
                    self.sigma, self.delta, self.L_chol, self.Y_init,
                    self.T, self.dt_max, i
                )
                results[i, 0] = total_loss
                results[i, 1] = L_weight
        
        total_losses = results[:, 0]
        weights = results[:, 1]
        weighted_losses = total_losses * weights
        
        expected_loss = np.mean(weighted_losses)
        std_error = np.std(weighted_losses) / np.sqrt(M)
        elapsed_time = time.time() - start_time
        
        return {
            'expected_loss': expected_loss,
            'std_error': std_error,
            'ci_95': (expected_loss - 1.96*std_error, expected_loss + 1.96*std_error),
            'total_losses': total_losses,
            'weights': weights,
            'weighted_losses': weighted_losses,
            'elapsed_time': elapsed_time,
            'speed': M/elapsed_time
        }

In [41]:
test_params = draw_params(N=50, J=3)



#optimized PlainJAM
simulator_test = OptimizedJAMSimulator(test_params, T=1.0)
results_test = simulator_test.run_simulation(M=1000)
print(f"Plain JAM Results:")
print(f"Expected Loss = {results_test['expected_loss']}")
print(f"Standard Error = {results_test['std_error']:.4f}")
print(f"95% CI = {results_test['ci_95']}")
print(f"Total Losses: {results_test['total_losses'][:10]}...")
print(f"Weights: {results_test['weights'][:10]}...")
print(f"Weighted Losses: {results_test['weighted_losses'][:10]}...")

print(f"Speed: {results_test['speed']:.2f} simulations/second")


Plain JAM Results:
Expected Loss = 33.91586590873859
Standard Error = 0.4543
95% CI = (np.float64(33.02546407455641), np.float64(34.80626774292077))
Total Losses: [33. 15. 26. 10.  5. 23. 18. 26. 11.  7.]...
Weights: [1.91500822 2.41567385 1.87709484 1.60971818 1.70383042 1.56438174
 0.85080696 1.75880967 1.46500413 1.46904582]...
Weighted Losses: [63.19527122 36.23510781 48.80446578 16.09718182  8.51915211 35.98078006
 15.31452525 45.72905142 16.11504545 10.28332075]...
Speed: 774.66 simulations/second
